# Missing value imputation: DropMissingData

Deletes rows with missing values.

DropMissingData works both with numerical and categorical variables.

**For this demonstration, we use the Ames House Prices dataset produced by Professor Dean De Cock:**

[Dean De Cock (2011) Ames, Iowa: Alternative to the Boston Housing
Data as an End of Semester Regression Project, Journal of Statistics Education, Vol.19, No. 3](http://jse.amstat.org/v19n3/decock.pdf)

The version of the dataset used in this notebook can be obtained from [Kaggle](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data)

## Version

In [1]:
# Make sure you are using this 
# Feature-engine version.

import feature_engine

feature_engine.__version__

'1.6.2'

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

from feature_engine.imputation import DropMissingData

In [ ]:
# # Download the data from Kaggle and store it in the same folder as this notebook.
# data = pd.read_csv('../data/housing.csv')
# data.head()

# # Separate the data into train and test sets.
# X_train, X_test, y_train, y_test = train_test_split(
#     data.drop(['Id', 'SalePrice'], axis=1),
#     data['SalePrice'],
#     test_size=0.3,
#     random_state=0,
# )

# X_train.shape, X_test.shape

In [4]:
# Read the separate files
train_df = pd.read_csv('../data/house-prices/train.csv')
test_df = pd.read_csv('../data/house-prices/test.csv')

# Separate features and target in training data
X_train = train_df.drop(['Id', 'SalePrice'], axis=1)
y_train = train_df['SalePrice']

# For test data, you might not have the target variable
X_test = test_df.drop(['Id'], axis=1)  # Note: test data might not have SalePrice column

print("X_train :", X_train.shape)
print("X_test :", X_test.shape)

X_train : (1460, 79)
X_test : (1459, 79)


## Drop data based on specific variables.

We can drop observations that show NA in any of a subset of variables.

In [5]:
# Drop data when there are NA in any of the indicated variables

imputer = DropMissingData(
    variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'],
    missing_only=False,
)

In [6]:
imputer.fit(X_train)

DropMissingData(missing_only=False,
                variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'])

In [7]:
# variables from which observations with NA will be deleted

imputer.variables_

['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea']

In [8]:
# Number of observations with NA before the transformation

X_train[imputer.variables].isna().sum()

Alley          1369
MasVnrType      872
LotFrontage     259
MasVnrArea        8
dtype: int64

In [9]:
# After the transformation the rows with NA values are 
# deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

In [10]:
# Number of observations with NA after transformation

train_t[imputer.variables].isna().sum()

Alley          0
MasVnrType     0
LotFrontage    0
MasVnrArea     0
dtype: int64

In [11]:
# Shape of dataframe before transformation

X_train.shape

(1460, 79)

In [12]:
# Shape of dataframe after transformation

train_t.shape

(13, 79)

In [13]:
# The "return_na_data()" method, returns a dataframe that contains
# the observations with NA. 

# That is, the portion of the data that is dropped when
# we apply the transform() method.

tmp = imputer.return_na_data(X_train)

tmp.shape

(1447, 79)

In [14]:
# total obs - obs with NA = final dataframe shape
#  after the transformation

1022-963

59

Sometimes, it is useful to retain the observation with NA in the production environment, to log which
observations are not being scored by the model for example.

## Drop data when variables contain %  of NA

We can drop observations if they contain less than a required percentage of values in a subset of observations.

In [15]:
# Drop data if an observation contains NA in 
# 2 of the 4 indicated variables (50%).

imputer = DropMissingData(
    variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'],
    missing_only=False,
    threshold=0.5,
)

In [16]:
imputer.fit(X_train)

DropMissingData(missing_only=False, threshold=0.5,
                variables=['Alley', 'MasVnrType', 'LotFrontage', 'MasVnrArea'])

In [17]:
# After the transformation the rows with NA values are 
# deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

In [18]:
# Number of observations with NA after transformation

train_t[imputer.variables].isna().sum()

Alley          1231
MasVnrType      734
LotFrontage     126
MasVnrArea        1
dtype: int64

We see that not all missing observations were dropped, because we required the observation to have NA in more than 1 of the variables at the time. 

## Automatically select all variables

We can drop obserations if they show NA in any variable in the dataset.

When the parameter `variables` is left to None and the parameter `missing_only` is left to True, the imputer will evaluate observations based of all variables with missing data.

When the parameter `variables` is left to None and the parameter `missing_only` is switched to False, the imputer will evaluate observations based of all variables.

It is good practice to use `missing_only=True` when we set `variables=None`, so that the transformer handles the imputation automatically in a meaningful way.

### Automatically find variables with NA

In [19]:
# Find variables with NA

imputer = DropMissingData(missing_only=True)

imputer.fit(X_train)

DropMissingData()

In [20]:
# variables with NA in the train set

imputer.variables_

['LotFrontage',
 'Alley',
 'MasVnrType',
 'MasVnrArea',
 'BsmtQual',
 'BsmtCond',
 'BsmtExposure',
 'BsmtFinType1',
 'BsmtFinType2',
 'Electrical',
 'FireplaceQu',
 'GarageType',
 'GarageYrBlt',
 'GarageFinish',
 'GarageQual',
 'GarageCond',
 'PoolQC',
 'Fence',
 'MiscFeature']

In [21]:
# Number of observations with NA

X_train[imputer.variables_].isna().sum()

LotFrontage      259
Alley           1369
MasVnrType       872
MasVnrArea         8
BsmtQual          37
BsmtCond          37
BsmtExposure      38
BsmtFinType1      37
BsmtFinType2      38
Electrical         1
FireplaceQu      690
GarageType        81
GarageYrBlt       81
GarageFinish      81
GarageQual        81
GarageCond        81
PoolQC          1453
Fence           1179
MiscFeature     1406
dtype: int64

In [22]:
# After the transformation the rows with NA are deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

In [23]:
# Number of observations with NA after the transformation

train_t[imputer.variables_].isna().sum()

LotFrontage     0.0
Alley           0.0
MasVnrType      0.0
MasVnrArea      0.0
BsmtQual        0.0
BsmtCond        0.0
BsmtExposure    0.0
BsmtFinType1    0.0
BsmtFinType2    0.0
Electrical      0.0
FireplaceQu     0.0
GarageType      0.0
GarageYrBlt     0.0
GarageFinish    0.0
GarageQual      0.0
GarageCond      0.0
PoolQC          0.0
Fence           0.0
MiscFeature     0.0
dtype: float64

In [24]:
# in this case, all observations will be dropped
# because all of them show NA at least in 1 variable

train_t.shape

(0, 79)

## Drop rows with % of missing data

Not to end up with an empty dataframe, let's drop rows that have less than 75% of the variables with values.

In [25]:
# Find variables with NA

imputer = DropMissingData(
    missing_only=True,
    threshold=0.75,
)

imputer.fit(X_train)

DropMissingData(threshold=0.75)

In [26]:
# After the transformation the rows with NA are deleted form the dataframe

train_t = imputer.transform(X_train)
test_t = imputer.transform(X_test)

In [27]:
train_t.shape

(1460, 79)

Now, we do have some data left.